# Week 3 - Data Manipulation

In [1]:
import h2o
import pandas as pd
import numpy as np

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_232"; OpenJDK Runtime Environment (build 1.8.0_232-8u232-b09-0ubuntu1~19.04.1-b09); OpenJDK 64-Bit Server VM (build 25.232-b09, mixed mode)
  Starting server from /home/megan/Projects/h2oclass/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp1cf5zt06
  JVM stdout: /tmp/tmp1cf5zt06/h2o_megan_started_from_python.out
  JVM stderr: /tmp/tmp1cf5zt06/h2o_megan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_megan_bvn2lk
H2O cluster total nodes:,1
H2O cluster free memory:,1.520 Gb
H2O cluster total cores:,3
H2O cluster allowed cores:,3
H2O cluster status:,"accepting new members, healthy"


In [3]:
# import the airlines data again
data = h2o.import_file('http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
train, valid, test = data.split_frame([0.8,0.1], seed=69)

In [6]:
print('%d/%d/%d' % (train.nrows, valid.nrows, test.nrows))

35255/4272/4451


In [7]:
# get the first group without using random sampling and assign a name
train2 = data[:35255,:]
train2 = h2o.assign(train2, "first35255")

In [8]:
# what do we have in h2o - use the ls command
h2o.ls()

,key
0,allyears2k_headers.hex
1,first35255
2,py_3_sid_b108
3,py_4_sid_b108
4,py_5_sid_b108


In [9]:
data.ncol

31

In [10]:
# create frame of dates
dates = data[:, :4]

In [11]:
# create frame of airports
airports = data[:, ['Origin', 'Dest']]

In [12]:
dates.ncol

4

In [13]:
airports.ncol

2

In [14]:
# use cbind to join (bind) columns together
a_and_d = airports.cbind(dates)

In [15]:
a_and_d.dim

[43978, 6]

In [16]:
# use rbind to join (bind) rows together
restored_data = train.rbind([valid, test])

In [17]:
restored_data.dim

[43978, 31]

In [18]:
# check that the dimensions of restored_data match that of the original data
data.dim

[43978, 31]

Review the heads of the dataframes

In [19]:
data[:, :4].head()

Year,Month,DayofMonth,DayOfWeek
1987,10,14,3
1987,10,15,4
1987,10,17,6
1987,10,18,7
1987,10,19,1
1987,10,21,3
1987,10,22,4
1987,10,23,5
1987,10,24,6
1987,10,25,7


In [20]:
train[:, :4].head()

Year,Month,DayofMonth,DayOfWeek
1987,10,14,3
1987,10,15,4
1987,10,17,6
1987,10,18,7
1987,10,22,4
1987,10,23,5
1987,10,24,6
1987,10,25,7
1987,10,26,1
1987,10,28,3


Merging Examples!

In [21]:
A = h2o.H2OFrame(
    pd.DataFrame({
        'a': range(1,11),
        'b': range(11,21),
        'c': range(21,31)
    }),
    destination_frame = 'A'
)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [22]:
A.as_data_frame()

,a,b,c
0,1,11,21
1,2,12,22
2,3,13,23
3,4,14,24
4,5,15,25
5,6,16,26
6,7,17,27
7,8,18,28
8,9,19,29
9,10,20,30


In [23]:
np.random.seed(123)
B = h2o.H2OFrame(
    pd.DataFrame({
        'a': np.random.randint(6,13,20),
        'd': np.random.randint(6,13,20)
    }),
    destination_frame = 'B'
)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
B.as_data_frame()

,a,d
0,12,6
1,11,7
2,12,9
3,8,12
4,10,11
5,8,10
6,12,6
7,7,6
8,9,10
9,8,7


In [25]:
M = A.merge(B)

In [26]:
M.as_data_frame()

,a,b,c,d
0,6,16,26,10
1,6,16,26,7
2,7,17,27,6
3,7,17,27,8
4,7,17,27,8
5,7,17,27,6
6,7,17,27,6
7,8,18,28,12
8,8,18,28,10
9,8,18,28,7


In [27]:
M = B.merge(A)

In [28]:
M.as_data_frame()

,a,d,b,c
0,6,10,16,26
1,6,7,16,26
2,7,6,17,27
3,7,8,17,27
4,7,8,17,27
5,7,6,17,27
6,7,6,17,27
7,8,12,18,28
8,8,10,18,28
9,8,7,18,28


In [30]:
M = B.merge(A, by_x='d', by_y=['a'])

In [31]:
M.as_data_frame()

,d,a,b,c
0,6,12,16,26
1,6,12,16,26
2,6,7,16,26
3,6,7,16,26
4,6,7,16,26
5,7,11,17,27
6,7,8,17,27
7,7,6,17,27
8,8,7,18,28
9,8,7,18,28


In [32]:
M = B.merge(A, by_y='b', by_x=['a'])

In [33]:
M.as_data_frame()

,a,d,a.1,c
0,11,7,1,21
1,12,6,2,22
2,12,9,2,22
3,12,6,2,22
4,12,10,2,22
5,12,11,2,22
6,12,9,2,22


In [34]:
M = A.merge(B, by_y='a', by_x=['a'], all_x=True)

In [35]:
M.as_data_frame()

,a,b,c,d
0,1,11,21,NaN
1,2,12,22,NaN
2,3,13,23,NaN
3,4,14,24,NaN
4,5,15,25,NaN
5,6,16,26,10.0
6,6,16,26,7.0
7,7,17,27,6.0
8,7,17,27,8.0
9,7,17,27,8.0


In [36]:
M = A.merge(B, all_y=True)

In [37]:
M.as_data_frame()

,a,b,c,d
0,6,16.0,26.0,10
1,6,16.0,26.0,7
2,7,17.0,27.0,6
3,7,17.0,27.0,8
4,7,17.0,27.0,8
5,7,17.0,27.0,6
6,7,17.0,27.0,6
7,8,18.0,28.0,12
8,8,18.0,28.0,10
9,8,18.0,28.0,7
